In [ ]:
import sys
sys.path.insert(0,'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
import cv2
import torchvision
from torch import nn
import numpy as np
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms
from torch.autograd import Variable
import os
import pandas as pd
import seaborn as sns
from dataloaders import get_train_transforms, get_val_transforms, get_triplet_dataloader
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from sklearn.metrics import accuracy_score, f1_score , precision_score , recall_score
from sklearn.manifold import TSNE
path_data = 'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
device = torch.cuda.is_available()
# استخراج ویژگی‌ها با استفاده از مدل Siamese
 
def generate_embeddings(data_loader, model):
    with torch.no_grad():
        #device = 'cuda'
        model.eval()
        #model.to(device)
        embeddings = []
        labels = []
        for batch_imgs, batch_labels in data_loader:
            if device:
                batch_imgs = batch_imgs.cuda()
            batch_E = model.get_embedding(batch_imgs)
            embeddings.append(batch_E.cpu().numpy())
            labels.append(batch_labels.numpy())
    return np.concatenate(embeddings), np.concatenate(labels)
 

# ذخیره مدل
#torch.save(siamese_model.state_dict(), "f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/siamese_model_EmbeddingGEnerator_State_improved.h5")
#torch.save(siamese_model , "f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/siamese_model_EmbeddingGEnerator.h5")

siamese_model = torch.load("f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/siamese_model_EmbeddingGEnerator.h5",map_location=torch.device('cuda'))
siamese_model.eval() 
print("siamese_model")
# تعریف مدل سفارشی ViT
class RefinedViT(nn.Module):
    def __init__(self, original_vit_model,embedding_dim,  num_classes):
        super(RefinedViT, self).__init__()
        # حفظ لایه‌های اصلی ViT
        self.num_patches = (224 // 16) * (224 // 16)  # برای تصویر 224x224 با پچ 16x16
        self.embed_dim = 768  # سایز استاندارد برای vit-base
        
        # تبدیل embedding به پچ‌ها
        self.embedding_to_patch = nn.Linear(1792, self.num_patches * self.embed_dim)
       # self.embedding_to_patch = nn.Linear(1280, self.num_patches * self.embed_dim)
        #self.embedding_to_patch = nn.Linear(embedding_dim,  self.num_patches * self.embed_dim)
        
        # Position Embedding
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.embed_dim))
        
        # CLS Token
        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embed_dim,
            nhead=12,  # تعداد heads در self-attention
            dim_feedforward=3072,
            dropout=0.1,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=12)
        
        # لایه‌های نهایی
        self.norm = nn.LayerNorm(self.embed_dim)
        self.fc = nn.Linear(self.embed_dim, num_classes)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        batch_size = x.size(0)
        
        # تبدیل embedding به پچ‌ها
        x = self.embedding_to_patch(x)  # (batch_size, num_patches * embed_dim)
        x = x.view(batch_size, self.num_patches, self.embed_dim)  # (batch_size, num_patches, embed_dim)
       
        # اضافه کردن CLS token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)  # (batch_size, num_patches + 1, embed_dim)
        
        # اضافه کردن position embedding
        x = x + self.pos_embed
        
        # Transformer Encoder
        x = self.transformer_encoder(x)
        
        # استفاده از CLS token برای طبقه‌بندی
        x = x[:, 0]  # فقط CLS token
        x = self.norm(x)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x  
    def get_embedding(self, x):
        """
        دریافت embedding برای استفاده در t-SNE
        """
        batch_size = x.size(0)
        
        # تغییر شکل ورودی
        if len(x.shape) == 4:
            x = x.view(batch_size, -1)  # تبدیل به بردار یک‌بعدی
        
        # تغییر ابعاد برای تطابق با عملیات ضرب ماتریسی
        x = x.reshape(batch_size, 1792, -1)  # تقسیم 150528 به 1792 و باقیمانده
        x = x.mean(dim=-1)  # میانگین‌گیری روی بعد آخر
        
        # حالا x دارای ابعاد (batch_size, 1792) است
        x = self.embedding_to_patch(x)  # عملیات خطی
        x = x.view(batch_size, self.num_patches, self.embed_dim)
        
        # اضافه کردن CLS token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        # اضافه کردن position embedding
        x = x + self.pos_embed
        
        # Transformer Encoder
        x = self.transformer_encoder(x)
        
        # استفاده از CLS token
        x = x[:, 0]
        x = self.norm(x)
        
        return x
     

train_data = torchvision.datasets.ImageFolder(root=path_data + '/train/', transform=get_val_transforms())
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32)

test_data = torchvision.datasets.ImageFolder(root=path_data + '/test/', transform=get_val_transforms())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

# بارگذاری مدل ViT از Hugging Face
vit_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
vit_model.classifier = torch.nn.Linear(vit_model.config.hidden_size, 15)  # تعداد کلاس‌ها

embedding_dim = vit_model.config.hidden_size  # ابعاد embedding
model = RefinedViT(vit_model, embedding_dim, num_classes=15)
print("model")
# استخراج ویژگی‌ها از داده‌های آموزش
train_embeddings, train_labels = generate_embeddings(train_loader, siamese_model )
# استخراج ویژگی‌ها از داده‌های تست
test_embeddings, test_labels = generate_embeddings(test_loader, siamese_model )
 
# تبدیل داده‌ها به تنسور
X_train, y_train = train_embeddings, train_labels
X_val, y_val = test_embeddings, test_labels

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# انتقال مدل به دستگاه CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.cuda.is_available():
    model.cuda()

# تنظیمات loss function و optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)  # استفاده از AdamW

# انتقال به GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# تنظیمات آموزش
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000)
print("train")
# آموزش
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')
 

torch.save(model,"f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/RVIT_improved.h5");
 
# ارزیابی مدل
model.eval()
with torch.no_grad():
    val_outputs = model(X_val_tensor.to(device))
    _, predicted = torch.max(val_outputs.data, 1)

# ذخیره مدل
torch.save(model.state_dict(), "f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/RVIT_State_improved.h5")
 


# محاسبه دقت و امتیاز F1
accuracy = accuracy_score(y_val_tensor.cpu(), predicted.cpu())
f1 = f1_score(y_val_tensor.cpu(), predicted.cpu(), average='weighted')
precision = precision_score(y_val_tensor.cpu(), predicted.cpu(), average='macro') 
recall = recall_score(y_val_tensor.cpu(), predicted.cpu(), average='macro')
